In [23]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

In [25]:
mydata = load_dataset("glue", "mrpc")
mydata

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [26]:
print(mydata["test"][:5])

mydata["train"].features

{'sentence1': ["PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .", "The world 's two largest automakers said their U.S. sales declined more than predicted last month as a late summer sales frenzy caused more of an industry backlash than expected .", 'According to the federal Centers for Disease Control and Prevention ( news - web sites ) , there were 19 reported cases of measles in the United States in 2002 .', 'A tropical storm rapidly developed in the Gulf of Mexico Sunday and was expected to hit somewhere along the Texas or Louisiana coasts by Monday night .', "The company didn 't detail the costs of the replacement and repairs ."], 'sentence2': ['Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .', 'Domestic sales at both GM and No. 2 Ford Motor Co. declined more than predicted as a late summer sales frenzy prompted a larger-than-expected industr

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [27]:
checkpoint = "bert-base-uncased"

tokenizer  = AutoTokenizer.from_pretrained(checkpoint)

In [28]:
def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = mydata.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [29]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1","sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [30]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [31]:
# tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["labels"],
#     shuffle=True,
#     collate_fn=data_collator,
#     batch_size=8,
# )

# tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["labels"],
#     shuffle=True,
#     collate_fn=data_collator,
#     batch_size=8,
# )

# tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["labels"],
#     shuffle=True,
#     collate_fn=data_collator,
#     batch_size=8,
# )

tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [32]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

loss = SparseCategoricalCrossentropy(from_logits=True)

num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)


opt = Adam(learning_rate=lr_scheduler)

In [34]:
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model.fit(tf_train_dataset, epochs=3, batch_size=8, validation_data=tf_validation_dataset)

Epoch 1/3
458/458 [==============================] - 129s 192ms/step - loss: 0.5471 - accuracy: 0.7194 - val_loss: 0.4356 - val_accuracy: 0.7868
Epoch 2/3
458/458 [==============================] - 78s 169ms/step - loss: 0.2986 - accuracy: 0.8753 - val_loss: 0.3623 - val_accuracy: 0.8358
Epoch 3/3
458/458 [==============================] - 75s 164ms/step - loss: 0.0968 - accuracy: 0.9703 - val_loss: 0.4070 - val_accuracy: 0.8505


In [35]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Evaluating Model**

In [36]:
import evaluate

model.evaluate(tf_validation_dataset)

51/51 [==============================] - 3s 60ms/step - loss: 0.4070 - accuracy: 0.8505


[0.4070189595222473, 0.8504902124404907]

In [37]:
model.evaluate(tf_test_dataset)

216/216 [==============================] - 13s 60ms/step - loss: 0.4058 - accuracy: 0.8487


[0.40582215785980225, 0.8486956357955933]

**Making Predictions and checking metrics**

In [38]:
validation_preds  = model.predict(tf_validation_dataset)["logits"]

51/51 [==============================] - 6s 56ms/step


In [39]:
probabilities = tf.nn.softmax(validation_preds)
validation_class_preds = np.argmax(probabilities, axis=1)

validation_class_preds

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,

In [40]:
print(confusion_matrix(validation_class_preds, mydata["validation"]["label"]))

print(classification_report(validation_class_preds, mydata["validation"]["label"]))

[[ 90  22]
 [ 39 257]]
              precision    recall  f1-score   support

           0       0.70      0.80      0.75       112
           1       0.92      0.87      0.89       296

    accuracy                           0.85       408
   macro avg       0.81      0.84      0.82       408
weighted avg       0.86      0.85      0.85       408



In [41]:
test_preds  = model.predict(tf_test_dataset)["logits"]

216/216 [==============================] - 13s 58ms/step


In [42]:
test_probabilities = tf.nn.softmax(test_preds)
test_class_preds = np.argmax(test_probabilities, axis=1)


In [43]:
print(confusion_matrix(test_class_preds, mydata["test"]["label"]))

print(classification_report(test_class_preds, mydata["test"]["label"]))

[[166 351]
 [412 796]]
              precision    recall  f1-score   support

           0       0.29      0.32      0.30       517
           1       0.69      0.66      0.68      1208

    accuracy                           0.56      1725
   macro avg       0.49      0.49      0.49      1725
weighted avg       0.57      0.56      0.56      1725



**GLUE benchmark Metrics**

In [44]:
from datasets import load_metric
metric = load_metric("glue", "mrpc")

metric.compute(predictions=validation_class_preds, references=np.array(mydata["validation"]["label"]))

{'accuracy': 0.8504901960784313, 'f1': 0.8939130434782607}